In [ ]:
# Kubiya Workflow SDK - Getting Started

This notebook provides a comprehensive introduction to the Kubiya Workflow SDK, including:
- Basic workflow creation
- Streaming execution
- AI-powered workflow generation with ADK provider
- Testing and debugging workflows

## Prerequisites

Make sure you have:
1. Installed the SDK: `pip install kubiya-workflow-sdk[all]`
2. Set your API keys:
   - `KUBIYA_API_KEY` - Get from [app.kubiya.ai](https://app.kubiya.ai)
   - `TOGETHER_API_KEY` - Get from [together.ai](https://together.ai) (for AI features)


In [ ]:
# Import required modules
import os
import json
from kubiya_workflow_sdk import Workflow, Step, Client
from kubiya_workflow_sdk.dsl import workflow
from kubiya_workflow_sdk.providers import get_provider

# Check if API keys are set
api_key_status = "✅" if os.getenv("KUBIYA_API_KEY") else "❌"
together_key_status = "✅" if os.getenv("TOGETHER_API_KEY") else "❌"

print(f"KUBIYA_API_KEY: {api_key_status}")
print(f"TOGETHER_API_KEY: {together_key_status}")

# Initialize client
client = Client()
print(f"\nConnected to Kubiya organization: {client.get_organization_info()['name']}")


In [ ]:
## 1. Creating Your First Workflow

Let's start with a simple workflow that demonstrates the basic concepts:


In [ ]:
# Method 1: Object-oriented approach
hello_workflow = Workflow(
    name="hello-world",
    description="A simple hello world workflow",
    steps=[
        Step(
            name="greeting",
            command="echo 'Hello from Kubiya!'"
        ),
        Step(
            name="show_date",
            command="date",
            depends=["greeting"]
        ),
        Step(
            name="final_message",
            command="echo 'Workflow completed successfully!'",
            depends=["show_date"]
        )
    ]
)

print("Workflow created:")
print(f"- Name: {hello_workflow.name}")
print(f"- Steps: {len(hello_workflow.steps)}")
print(f"- Dependencies: {[s.depends for s in hello_workflow.steps if s.depends]}")


In [ ]:
# Method 2: DSL (Domain Specific Language) approach - more fluent and readable
hello_workflow_dsl = (
    workflow("hello-world-dsl")
    .description("A simple hello world workflow using DSL")
    .step("greeting", "echo 'Hello from Kubiya DSL!'")
    .step("show_date", "date", depends=["greeting"])
    .step("show_user", "echo 'Running as user: $(whoami)'", depends=["greeting"])
    .step("final_message", "echo 'All steps completed!'", depends=["show_date", "show_user"])
    .build()
)

# Visualize the workflow structure
print("DSL Workflow structure:")
for step in hello_workflow_dsl.steps:
    deps = f" (depends on: {', '.join(step.depends)})" if step.depends else ""
    print(f"  - {step.name}: {step.command[:50]}...{deps}")


In [ ]:
## 2. Executing Workflows

Let's execute our workflow and see it in action. We'll use streaming to get real-time updates:


In [ ]:
# Execute with streaming for real-time updates
print("🚀 Executing workflow with streaming...\n")

try:
    for event in client.execute_workflow(hello_workflow_dsl, stream=True):
        if hasattr(event, 'type'):
            if event.type == "step_started":
                print(f"▶️  Starting step: {event.step_name}")
            elif event.type == "step_completed":
                print(f"✅ Completed step: {event.step_name}")
                if hasattr(event, 'output') and event.output:
                    print(f"   Output: {event.output.strip()}")
            elif event.type == "workflow_completed":
                print(f"\n🎉 Workflow completed with status: {event.status}")
                print(f"   Execution ID: {event.execution_id}")
        else:
            # Handle other event formats
            print(f"   Event: {event}")
            
except Exception as e:
    print(f"❌ Error executing workflow: {e}")
    print("Note: Make sure your API key is valid and not expired")


In [ ]:
## 3. Advanced Workflow Features

Let's create a more complex workflow that demonstrates:
- Parameters
- Error handling and retries
- Conditional execution
- Output capture


In [ ]:
# Create an advanced deployment workflow
advanced_workflow = (
    workflow("advanced-deployment")
    .description("Advanced deployment workflow with error handling")
    .params(
        APP_NAME="${APP_NAME}",
        VERSION="${VERSION:-latest}",  # Default value
        ENVIRONMENT="${ENVIRONMENT:-staging}",
        MAX_RETRIES="3"
    )
    
    # Step 1: Validate inputs
    .step("validate")
    .shell("""
        if [ -z "$APP_NAME" ]; then
            echo "ERROR: APP_NAME is required"
            exit 1
        fi
        echo "Validated: Deploying $APP_NAME:$VERSION to $ENVIRONMENT"
    """)
    
    # Step 2: Check prerequisites with retry
    .step("check_health")
    .shell("""
        echo "Checking system health..."
        # Simulate health check (in real world, this would check actual services)
        if [ "$RANDOM" -gt 20000 ]; then
            echo "Health check passed"
            exit 0
        else
            echo "Health check failed, will retry..."
            exit 1
        fi
    """)
    .retry(limit=3, interval_sec=5)
    .depends(["validate"])
    
    # Step 3: Deploy application
    .step("deploy")
    .shell("""
        echo "Deploying $APP_NAME version $VERSION..."
        echo "Target environment: $ENVIRONMENT"
        sleep 2
        echo "Deployment completed"
    """)
    .output("DEPLOYMENT_ID")  # Capture output
    .depends(["check_health"])
    
    # Step 4: Run tests
    .step("test")
    .shell("""
        echo "Running smoke tests..."
        echo "Testing endpoint: http://$APP_NAME.$ENVIRONMENT.local/health"
        echo "All tests passed!"
    """)
    .depends(["deploy"])
    .continue_on(failure=True)  # Continue even if tests fail
    
    # Step 5: Notification
    .step("notify")
    .shell("""
        status="SUCCESS"
        if [ "$test_exit_code" != "0" ]; then
            status="FAILED"
        fi
        echo "Deployment notification:"
        echo "- App: $APP_NAME"
        echo "- Version: $VERSION"
        echo "- Environment: $ENVIRONMENT"
        echo "- Status: $status"
        echo "- Deployment ID: ${DEPLOYMENT_ID:-unknown}"
    """)
    .depends(["test"])
    
    .build()
)

# Display workflow info
print("Advanced Workflow Configuration:")
print(f"- Name: {advanced_workflow.name}")
print(f"- Parameters: {list(advanced_workflow.params.keys())}")
print(f"- Steps with retry: {[s.name for s in advanced_workflow.steps if s.retry]}")
print(f"- Steps with output: {[s.name for s in advanced_workflow.steps if s.output]}")


In [ ]:
## 4. AI-Powered Workflow Generation with ADK

The ADK (Agent Development Kit) provider allows you to generate workflows from natural language descriptions. This is incredibly powerful for quickly creating complex workflows without writing code.


In [ ]:
# Initialize ADK provider
try:
    adk_provider = get_provider("adk")
    print("✅ ADK provider initialized successfully")
    print("   Using DeepSeek V3 models via Together AI")
except Exception as e:
    print(f"❌ Failed to initialize ADK provider: {e}")
    print("   Make sure TOGETHER_API_KEY is set")
    adk_provider = None


In [ ]:
# Generate a workflow from natural language (async example converted to sync for notebook)
import asyncio

async def generate_backup_workflow():
    """Generate a database backup workflow using AI"""
    
    task_description = """
    Create a workflow that:
    1. Backs up a PostgreSQL database named 'production_db'
    2. Compresses the backup file with gzip
    3. Uploads the backup to S3 bucket 'company-backups' with timestamp
    4. Verifies the upload was successful
    5. Removes local backup files older than 7 days
    6. Sends a Slack notification with backup details
    """
    
    print("🤖 Generating workflow from description...\n")
    
    # Generate workflow in plan mode (no execution)
    result = await adk_provider.compose(
        task=task_description,
        mode="plan",  # Just generate, don't execute
        stream=True
    )
    
    generated_workflow = None
    
    # Process the stream
    async for event in result:
        if event.get("type") == "text":
            # Show AI's thinking process
            content = event.get("content", "")
            if content and not content.startswith("SSE"):
                print(f"AI: {content[:100]}...")
        elif event.get("type") == "workflow":
            generated_workflow = event.get("data")
    
    return generated_workflow

# Run the async function
if adk_provider:
    try:
        generated_workflow = asyncio.run(generate_backup_workflow())
        
        if generated_workflow:
            print("\n✅ Workflow generated successfully!")
            print(f"\nGenerated workflow: {generated_workflow.name}")
            print(f"Description: {generated_workflow.description}")
            print(f"\nSteps ({len(generated_workflow.steps)}):")
            for step in generated_workflow.steps:
                deps = f" (depends on: {', '.join(step.depends)})" if step.depends else ""
                retry = f" [retry: {step.retry['limit']}x]" if step.retry else ""
                print(f"  - {step.name}{deps}{retry}")
                
            # Show the generated workflow as YAML
            print("\n" + "="*60)
            print("Generated Workflow YAML:")
            print("="*60)
            print(generated_workflow.to_yaml())
            
    except Exception as e:
        print(f"Error generating workflow: {e}")


In [ ]:
## 5. Working with Platform Resources

Kubiya provides various platform resources like runners, integrations, and secrets. Let's explore what's available:


In [ ]:
# Explore available platform resources
print("🔍 Exploring Kubiya Platform Resources\n")

# 1. List available runners
try:
    runners = client.get_runners()
    print("Available Runners:")
    for runner in runners[:5]:  # Show first 5
        print(f"  - {runner['name']}: {runner.get('description', 'No description')}")
    if len(runners) > 5:
        print(f"  ... and {len(runners) - 5} more")
except Exception as e:
    print(f"Error fetching runners: {e}")

print("\n" + "-"*60 + "\n")

# 2. List available integrations
try:
    integrations = client.get_integrations()
    print("Available Integrations:")
    for integration in integrations[:10]:  # Show first 10
        icon = integration.get('icon', '🔧')
        print(f"  {icon} {integration['name']}: {integration.get('description', '')[:50]}...")
    if len(integrations) > 10:
        print(f"  ... and {len(integrations) - 10} more")
except Exception as e:
    print(f"Error fetching integrations: {e}")

print("\n" + "-"*60 + "\n")

# 3. List available secrets (metadata only, not values)
try:
    secrets = client.get_secrets_metadata()
    print("Available Secrets:")
    for secret in secrets[:5]:  # Show first 5
        print(f"  - {secret['name']} (type: {secret.get('type', 'unknown')})")
    if len(secrets) > 5:
        print(f"  ... and {len(secrets) - 5} more")
except Exception as e:
    print(f"Error fetching secrets: {e}")

print("\n" + "-"*60 + "\n")

# 4. Get organization info
try:
    org_info = client.get_organization_info()
    print("Organization Info:")
    print(f"  - Name: {org_info.get('name', 'N/A')}")
    print(f"  - Plan: {org_info.get('plan', 'N/A')}")
    print(f"  - Created: {org_info.get('created_at', 'N/A')}")
except Exception as e:
    print(f"Error fetching organization info: {e}")


In [ ]:
## 6. Next Steps

Congratulations! You've learned the basics of the Kubiya Workflow SDK. Here's what you can explore next:

### 📚 Additional Resources

1. **Advanced Workflows**
   - Parallel execution
   - Sub-workflows
   - Complex conditionals
   - Integration with external tools

2. **Testing & Debugging**
   - Unit testing workflows
   - Mocking steps
   - Debugging failed executions

3. **Production Deployment**
   - Running the SDK server
   - Docker deployment
   - Kubernetes integration
   - CI/CD pipelines

### 🔗 Useful Links

- [Documentation](https://docs.kubiya.ai)
- [GitHub Repository](https://github.com/kubiya-ai/workflow-sdk)
- [Discord Community](https://discord.gg/kubiya)
- [API Reference](https://docs.kubiya.ai/api-reference)

### 💡 Tips

1. **Start Simple**: Begin with basic workflows and gradually add complexity
2. **Use AI Generation**: Let ADK provider generate complex workflows for you
3. **Test Locally**: Use the testing framework before deploying to production
4. **Monitor Executions**: Use streaming to monitor long-running workflows
5. **Leverage Integrations**: Use built-in integrations for common tools

Happy workflow building! 🚀
